In [38]:
import pandas as pd
import numpy as np
import json
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style
from matplotlib import pyplot
import seaborn as sns
import scipy.stats
from bs4 import BeautifulSoup
import re
import csv
import requests
from sklearn.ensemble import RandomForestClassifier
import textblob, string  
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn import svm
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from nltk import sent_tokenize
from nltk.corpus import stopwords
!pip install textblob
from textblob import TextBlob
# load functions from textpreprocess.py
from textpreprocess import denoise_text, normalize, replace_contractions, remove_non_ascii, to_lowercase, remove_punctuation, replace_numbers, remove_stopwords,lemmatize_verbs

In [2]:
def strip_url(text):
    new_text = re.sub(r"http.?://[^\s]+[\s]?", '', text)
    if new_text != '':
        return new_text
    else:
        return text

In [3]:
#function to convert to lowercase, remove punctuations, remove stopwords
def preprocess(text):
    text = replace_contractions(text)
    text = strip_url(text)
    content = nltk.word_tokenize(text)
    words = remove_non_ascii(content)
    words = to_lowercase(content)
    #words = lemmatize_verbs(words)
    words = remove_punctuation(content)
    words = replace_numbers(words)
    #words = remove_stopwords(words)
    text = ' '.join(words)
    return text

In [4]:
df = pd.read_csv('manually_labelled_tweets_for_modelling.csv',encoding='utf-8')

In [5]:
df.head()

,new_tweets,sentiment,classification
0,#Coronavirus: Greece shuts schools as WHO warn...,anger,reporting
1,âš ï¸The #coronavirus is spreading within com...,neutral,reporting
2,Clorox Healthcare Bleach Germicidal Cleaner Re...,neutral,marketing
3,Diamond Princess reports bringing total confir...,neutral,reporting
4,Microsoft asks employees in US to work from ho...,anger,reporting


In [6]:
df['new_tweets'] =  df['new_tweets'].apply(preprocess)

In [7]:
df.head()

,new_tweets,sentiment,classification
0,Coronavirus Greece shuts schools as WHO warns ...,anger,reporting
1,âš ïThe coronavirus is spreading within commun...,neutral,reporting
2,Clorox Healthcare Bleach Germicidal Cleaner Re...,neutral,marketing
3,Diamond Princess reports bringing total confir...,neutral,reporting
4,Microsoft asks employees in US to work from ho...,anger,reporting


In [8]:
df['sentiment'].replace("neutral",0,inplace=True)
df['sentiment'].replace("anger",1,inplace=True)
df['sentiment'].replace("sadness",2,inplace=True)
df['sentiment'].replace("happy",3,inplace=True)
df['sentiment'].replace("hope",4,inplace=True)

In [9]:
df.head()

,new_tweets,sentiment,classification
0,Coronavirus Greece shuts schools as WHO warns ...,1,reporting
1,âš ïThe coronavirus is spreading within commun...,0,reporting
2,Clorox Healthcare Bleach Germicidal Cleaner Re...,0,marketing
3,Diamond Princess reports bringing total confir...,0,reporting
4,Microsoft asks employees in US to work from ho...,1,reporting


In [10]:
#from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# label encode the target variable 
#encoder = preprocessing.LabelEncoder()
#df['sentiment'] = encoder.fit_transform(df['sentiment'])
df['sentiment'].value_counts()

0    541
1    218
2    138
3     75
4     61
Name: sentiment, dtype: int64

In [11]:
df['classification'].value_counts()

opinion           489
reporting         349
precautions       106
joke               51
marketing          15
confusion          11
irrelevant          6
fakenews            4
discrimination      2
Name: classification, dtype: int64

In [12]:
df['classification'].replace("opinion",0,inplace=True)
df['classification'].replace("reporting",1,inplace=True)
df['classification'].replace("precautions",3,inplace=True)
df['classification'].replace("joke",4,inplace=True)
df['classification'].replace("marketing",5,inplace=True)
df['classification'].replace("confusion",6,inplace=True)
df['classification'].replace("irrelevant",7,inplace=True)
df['classification'].replace("fakenews",8,inplace=True)
df['classification'].replace("discrimination",9,inplace=True)

In [13]:
df['classification'].value_counts()

0    489
1    349
3    106
4     51
5     15
6     11
7      6
8      4
9      2
Name: classification, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [15]:
vectorizer = TfidfVectorizer()
final_features = vectorizer.fit_transform(df['new_tweets']).toarray()
final_features.shape

(1033, 4388)

In [16]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
    CountVectorizer(binary=True),
    TfidfTransformer(use_idf=True),
)
tfidf_train_data = pipe.fit_transform(df['new_tweets']) 
#tfidf_test_data = pipe.transform(testData)

In [17]:
y_train = df['sentiment']
y_train1 = df['classification']

In [18]:
tfidf_train_data.shape

(1033, 4388)

In [19]:
clf=RandomForestClassifier(n_estimators=100)
clf1=RandomForestClassifier(n_estimators=100)
clf.fit(tfidf_train_data, y_train) #sentiment
clf1.fit(tfidf_train_data, y_train1) #classification

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
def number_to_sentiment():
    df['sentiment'].replace(0,"neutral",inplace=True)
    df['sentiment'].replace(1,"anger",inplace=True)
    df['sentiment'].replace(2,"sadness",inplace=True)
    df['sentiment'].replace(3,"happy",inplace=True)
    df['sentiment'].replace(4,"hope",inplace=True)

def number_to_classification():
    df['classification'].replace(0,"opinion",inplace=True)
    df['classification'].replace(1,"reporting",inplace=True)
    df['classification'].replace(2,"precautions",inplace=True)
    df['classification'].replace(3,"joke",inplace=True)
    df['classification'].replace(4,"marketing",inplace=True)
    df['classification'].replace(5,"confusion",inplace=True)
    df['classification'].replace(6,"irrelevant",inplace=True)
    df['classification'].replace(7,"fakenews",inplace=True)
    df['classification'].replace(8,"discrimination",inplace=True)


In [32]:
csv_list = "NewCoronaTwitterData-BorisJohnson-UK.csv","NewCoronaTwitterData-EmmanuelMacron-Franc.csv","NewCoronaTwitterData-Jokowi-Indonasia.csv","NewCoronaTwitterData-JustinTrudeau-canada.csv","NewCoronaTwitterData-KingSalman-SaudiArab.csv","NewCoronaTwitterData-lopezobrador_mexico.csv","NewCoronaTwitterData-MohmamedBinZayed-UAE.csv","NewCoronaTwitterData-narendramodi.csv","NewCoronaTwitterData-Pontifex-pop.csv","NewCoronaTwitterData-realDonaldTrump.csv","NewCoronaTwitterData-leehsienloong-Sinagpore.csv"

In [33]:
for val in csv_list:
    df = pd.read_csv(val)
    df['tweets'] =  df['tweets'].apply(preprocess)
    df_data = pipe.transform(df['tweets'])
    senti_prediction = clf.predict(df_data)
    class_prediction = clf1.predict(df_data)
    a = senti_prediction.flatten()
    df['sentiment']=a
    b = class_prediction.flatten()
    df['classification']=b
    number_to_sentiment()
    number_to_classification()
    df.to_csv('Labelled'+val)

In [2]:
corona_csv_list = {"NewCoronaTwitterData-coronaviruscare.csv","NewCoronaTwitterDataCOVID-19.csv","NewCoronaTwitterData-Wuhan.csv","NewCoronaTwitterData-WuhanCoronavirus.csv","NewCoronaTwitterData-WuhanSARS.csv"}


In [25]:
for val in corona_csv_list:
    df = pd.read_csv(val)
    df['tweets'] =  df['tweets'].apply(preprocess)
    df_data = pipe.transform(df['tweets'])
    senti_prediction = clf.predict(df_data)
    class_prediction = clf1.predict(df_data)
    a = senti_prediction.flatten()
    df['sentiment']=a
    b = class_prediction.flatten()
    df['classification']=b
    number_to_sentiment()
    number_to_classification()
    df.to_csv('Labelled'+val)

In [ ]:
csv_list = "NewCoronaTwitterData-BorisJohnson-UK.csv","NewCoronaTwitterData-EmmanuelMacron-Franc.csv","NewCoronaTwitterData-Jokowi-Indonasia.csv","NewCoronaTwitterData-JustinTrudeau-canada.csv","NewCoronaTwitterData-KingSalman-SaudiArab.csv","NewCoronaTwitterData-lopezobrador_mexico.csv","NewCoronaTwitterData-MohmamedBinZayed-UAE.csv","NewCoronaTwitterData-narendramodi.csv","NewCoronaTwitterData-Pontifex-pop.csv","NewCoronaTwitterData-realDonaldTrump.csv","NewCoronaTwitterData-leehsienloong-Sinagpore.csv"

In [37]:
for val in csv_list:
    df = pd.read_csv(val)
    df['tweets'] =  df['tweets'].apply(preprocess)
    # print tweets
    # Use textblob to fetch sentimental of the tweet
    sentiment=[]
    sent_polarity = []
    sent_subjectivity = []
    for tweet in df['tweets']:
        analyse = TextBlob(tweet)
        sent_polarity.append(analyse.sentiment.polarity)
        sent_subjectivity.append(analyse.sentiment.subjectivity)
        if (analyse.sentiment.polarity > 0) :
            sentiment.append("positive")
        elif (analyse.sentiment.polarity < 0):
            sentiment.append("negative")
        else:
            sentiment.append("neutral")
#print(sentiment)
    df['sentiment'] = sentiment
    df.to_csv('Labelled_'+val)